In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
from datetime import datetime
import statsmodels.api as sm

# Use Kamppi as an example, could be generalized to all stations
df = pd.read_csv("datasets/Jämeräntaival_hourly_aggregate.csv")
df['Departure'] = pd.to_datetime(df['Departure'], format='mixed')
df.set_index('Departure', inplace=True)

train_end = datetime(year=2022, month=10, day=31)
train_start = datetime(year=2018,month=4,day=1)
train_data = df[:train_end]
test_end = datetime(year=2023, month=10, day = 31)
test_data  = df[datetime(year=2023,month=4,day=1):]

parameter_space = list()

for i in range(0,3):
    for j in range(0,3):
        for k in range(0,3):
            parameter_space.append((i,j,k))

#parameter_space = [x for x in parameter_space if x[1] == 0]
parameter_space = [x for x in parameter_space if x != (0,0,0)]
'''
param_mse = list()
for param in parameter_space:
    model = sm.tsa.statespace.SARIMAX(train_data['trip'], order=param, seasonal_order=(0, 1, 0, 24), freq='h').fit()
    forecast = model.forecast(datetime(year=2023,month=10,day=31,hour=17))
    forecast = forecast[datetime(year=2023,month=4,day=1):]
    #test_data.index = pd.DatetimeIndex(forecast.index)
    
    final = pd.concat([forecast,test_data], axis=1)
    errors = final['trip'] - final['predicted_mean']
    mse = np.mean(errors**2)

    param_mse.append((mse, param))
'''



    


"\nparam_mse = list()\nfor param in parameter_space:\n    model = sm.tsa.statespace.SARIMAX(train_data['trip'], order=param, seasonal_order=(0, 1, 0, 24), freq='h').fit()\n    forecast = model.forecast(datetime(year=2023,month=10,day=31,hour=17))\n    forecast = forecast[datetime(year=2023,month=4,day=1):]\n    #test_data.index = pd.DatetimeIndex(forecast.index)\n    \n    final = pd.concat([forecast,test_data], axis=1)\n    errors = final['trip'] - final['predicted_mean']\n    mse = np.mean(errors**2)\n\n    param_mse.append((mse, param))\n"

In [23]:
min_mse = param_mse[0][0]

index = 0
for i in range(0,len(param_mse)):
    if param_mse[i][0] < min_mse:
        print(param_mse[i][0], min_mse)
        min_mse = param_mse[i][0]
        index = i


print(param_mse[index])


9.043665073451029 9.056463656418249
9.029642126494705 9.043665073451029
9.029317575211266 9.029642126494705
(np.float64(9.029317575211266), (1, 0, 2))


In [5]:
model = sm.tsa.statespace.SARIMAX(train_data['trip'], order=(1,1,1), seasonal_order=(0,1,0,24), freq='h').fit()
forecast = model.forecast(datetime(year=2023,month=10,day=31,hour=17))
forecast = forecast[datetime(year=2023,month=4,day=1):]
#test_data.index = pd.DatetimeIndex(forecast.index)

final = pd.concat([forecast,test_data], axis=1)
errors = final['trip'] - final['predicted_mean']
mse = np.mean(errors**2)

print(mse)

/home/aleksi/venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency h will be used.
  self._init_dates(dates, freq)
 This problem is unconstrained.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            3     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  2.51523D+00    |proj g|=  8.46816D-02

At iterate    5    f=  2.46808D+00    |proj g|=  9.28777D-03

At iterate   10    f=  2.45763D+00    |proj g|=  8.55309D-03

At iterate   15    f=  2.45509D+00    |proj g|=  5.19865D-03

At iterate   20    f=  2.45471D+00    |proj g|=  1.64676D-03

At iterate   25    f=  2.45467D+00    |proj g|=  4.36219D-04

At iterate   30    f=  2.45467D+00    |proj g|=  2.78346D-06

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nac